In [1]:
from src import logger
from src.data.make_dataset import DataIngestion
from src.config.configuration import ConfigurationManager
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from src.utils.common import save_pickle, read_pickle
logger.name = "Feature Enginnering"

In [2]:
data_ingestion = DataIngestion()
config_manager = ConfigurationManager()
data_ingestion_params = config_manager.get_data_ingestion_config()

2025-01-08 14:11:12,919 - Feature Enginnering - INFO - Yaml read successfully from /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/config/config.yaml
2025-01-08 14:11:12,921 - Feature Enginnering - INFO - Yaml read successfully from /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/params.yaml


In [3]:
train = data_ingestion.get_data(data_ingestion_params.train_dir)

2025-01-08 14:11:12,944 - Feature Enginnering - INFO - Data read sucessfully ../data/interim/train/train.csv


In [4]:
train.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0508-SQWPL,Female,0,Yes,Yes,57,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),20.10,1087.7,No
1,3795-CAWEX,Male,0,Yes,Yes,70,Yes,Yes,Fiber optic,Yes,...,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),116.55,8152.3,No
2,0178-SZBHO,Male,0,Yes,Yes,47,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,No,Electronic check,87.20,4017.45,No
3,7137-NAXML,Male,0,No,No,23,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Bank transfer (automatic),20.15,405.6,No
4,3703-KBKZP,Male,1,No,No,16,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.45,1261.35,No


In [12]:
class FeatureEnginnering:
    def __init__(self):
        pass
    
    def check_null(self, data:pd.DataFrame,):
        try:
            if data.isnull().sum().sum() == 0:
                logger.debug("Data has no missing values")
            else:
                logger.warning("Data has missing values")
        except Exception as e:
            logger.error(e)
            
    def fit_label_encoder(self, data: pd.DataFrame):
        fitter = {}
        columns = data.select_dtypes(object).columns
        for col in columns:
            le = LabelEncoder()
            try:
                assert data[col].isnull().sum() == 0, logger.warning(f"Missing values occured in {col}")
                fitter.update({col: le.fit(data[col])})
            except Exception as e:
                logger.error(e)
        return fitter

    def encoder_transform(self, encoder, data):
        try:
            for col, ob in encoder.items():
                assert data[col].isnull().sum() ==0, logger.warning(f"Missing values occured in {col}")
                data[col] = ob.transform(data[col])
            return data
        except Exception as e:
            logger.error(e)
            


In [13]:
feature_eng = FeatureEnginnering()
fitter = feature_eng.fit_label_encoder(train)

In [7]:
config_manager.config

ConfigBox({'data_directory': {'root_dir': '../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv', 'interim_train': '../data/interim/train/train.csv', 'interim_test': '../data/interim/test/test.csv', 'processed_train': '../data/processed/train/train.csv', 'processed_test': '../data/processed/test/test.csv'}, 'artifacts': {'encoder': '../artifacts/encoder.pkl'}})

In [8]:
save_pickle(fitter, config_manager.config.artifacts.encoder)

2025-01-08 14:11:12,996 - Feature Enginnering - DEBUG - File path without extension is /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/artifacts
2025-01-08 14:11:12,997 - Feature Enginnering - INFO - Directory already present at /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/artifacts
2025-01-08 14:11:12,998 - Feature Enginnering - INFO - Deleting previous directory and creating directory at /Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/artifacts
2025-01-08 14:11:12,999 - Feature Enginnering - INFO - Pickle object stored at ../artifacts/encoder.pkl


In [9]:
fitter = read_pickle(config_manager.config.artifacts.encoder)

/Users/goldyrana/mess/deep_learning/projects/telco_customer_churn/artifacts/encoder.pkl


In [14]:
feature_eng.encoder_transform(fitter, train)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,289,0,0,1,1,57,1,0,2,1,...,1,1,1,1,2,1,0,20.10,134,0
1,2140,1,0,1,1,70,1,2,1,2,...,2,2,2,2,2,1,1,116.55,4925,0
2,85,1,0,1,1,47,1,2,1,0,...,0,0,2,0,0,0,2,87.20,2774,0
3,4070,1,0,0,0,23,1,0,2,1,...,1,1,1,1,1,0,0,20.15,2798,0
4,2074,1,1,0,0,16,1,2,1,0,...,0,0,0,0,0,1,0,74.45,378,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,4158,1,0,1,1,69,1,0,0,2,...,2,0,0,0,2,1,0,59.75,2811,0
5630,387,1,0,0,0,4,1,2,1,0,...,0,0,0,2,0,1,2,85.65,2370,1
5631,3029,1,0,0,0,35,1,2,1,0,...,2,2,2,0,0,0,2,95.50,2395,0
5632,3824,0,0,0,0,36,1,0,1,0,...,2,2,0,2,0,1,2,87.55,2158,1
